In [43]:
import json, requests 

syrisurl = "https://syris.iigweb.hevs.ch/"

mediaurl = syrisurl+"media/"

routesurl = syrisurl+"en/routes/route_data/2955"

url = requests.get(routesurl)

data = json.loads(url.text)

#user = data[0]
print(data['type'])

features = data['features']
#print(address)

route =[f for f in features if "duration" in f['properties']][0]
#print(route)

pods = [f for f in features if "img" in f['properties'] ]

print(len(pods))


FeatureCollection
19


In [44]:
from rdflib import Graph,Namespace
from rdflib import URIRef,BNode,Literal
from rdflib.namespace import RDF
from rdflib.namespace import RDFS, FOAF

from decimal import Decimal

In [45]:
ANNIVIERS= Namespace("https://hevs.ch/santour/anniviers/")
SANTOUR= Namespace("https://hevs.ch/ontology/santour#")
SCHEMA = Namespace("https://schema.org/")

route_prop=route['properties']

id=route['properties']['pk']

routeIri = URIRef(ANNIVIERS+id)


georoute=route['geometry']['coordinates']
gg=georoute[0]

ls=[[2,3],[4,5]]
lst=[str(i[0])+' '+str(i[1]) for i in ls]



coordlist = [str(coord[0])+' '+ str(coord[1]) for coord in gg]
wkt='LINESTRING('+ ', '.join(coordlist)+')'


g = Graph()


g.bind("schema",SCHEMA)
g.bind("santour",SANTOUR)
g.bind("anniviers",ANNIVIERS)


g.add( (routeIri, RDF.type, SANTOUR.Route ) )
g.add( (routeIri, SCHEMA.name, Literal(route_prop['title']) ) )
g.add( (routeIri, SCHEMA.identifier, Literal(route_prop['pk']) ) )
g.add( (routeIri, SCHEMA.url, URIRef(route_prop['subtitle']) ) )
g.add( (routeIri, SCHEMA.description, Literal(route_prop['description']) ) )
g.add( (routeIri, SANTOUR.duration, Literal(route_prop['duration']) ) )
g.add( (routeIri, SANTOUR.length, Literal(Decimal(route_prop['tour_length'])) ) )
g.add( (routeIri, SANTOUR.ascent, Literal(int(route_prop['ascent'])) ) )
g.add( (routeIri, SANTOUR.descent, Literal(int(route_prop['descent'])) ) )
g.add( (routeIri, SANTOUR.ibpIndex, Literal(int(route_prop['ibp_index'])) ) )
g.add( (routeIri, SANTOUR.effort, Literal(int(route_prop['energy_max'])) ) )
g.add( (routeIri, SANTOUR.technique, Literal(int(route_prop['technical_max'])) ) )
g.add( (routeIri, SANTOUR.risk, Literal(int(route_prop['risk_max'])) ) )
      
      
g.add( (routeIri, SCHEMA.geo, Literal(wkt) ) )

#'ibp_index': 54, 'energy_max': 3, 'technical_max': 2, 'risk_max': 3

<Graph identifier=Nb9f8284a990c459dbf2e1c3f05146d9c (<class 'rdflib.graph.Graph'>)>

In [46]:
for pod in pods:
    pod_prop=pod['properties']

    podid=pod_prop['pk']

    podIri = URIRef(ANNIVIERS+podid)

    geopod=pod['geometry']['coordinates']
    print(geopod)
    
    g.add( (routeIri, SANTOUR.pointOfDifficulty, podIri ) )
    g.add( (podIri, SCHEMA.name, Literal(pod_prop['title']) ) )
    g.add( (podIri, SCHEMA.description, Literal(pod_prop['description']) ) )
    g.add( (podIri, SCHEMA.identifier, Literal(pod_prop['pk']) ) )
    g.add( (podIri, SANTOUR.risk, Literal(int(pod_prop['risk']) )) )
    g.add( (podIri, SANTOUR.technique, Literal(int(pod_prop['technical']) )) )
    g.add( (podIri, SCHEMA.photo, URIRef(mediaurl+pod_prop['img']) ) )
    g.add( (podIri, SCHEMA.latitude, Literal(geopod[1]) ) )
    g.add( (podIri, SCHEMA.longitude, Literal(geopod[0]) ) )
    
    


[7.597378, 46.220972]
[7.597417, 46.221491]
[7.597265, 46.221655]
[7.595294, 46.224873]
[7.594734, 46.225603]
[7.594499, 46.226389]
[7.594492, 46.226393]
[7.592985, 46.231578]
[7.592985, 46.231578]
[7.595892, 46.234217]
[7.595892, 46.234217]
[7.59758, 46.235929]
[7.599571, 46.234236]
[7.60775, 46.230357]
[7.60775, 46.230357]
[7.60999, 46.229348]
[7.609989, 46.22924]
[7.610163, 46.229118]
[7.611088, 46.228529]


In [47]:
from rdflib.serializer import Serializer

context = {"@vocab": "http://purl.org/dc/terms/", 
           "rdfs":"http://www.w3.org/2000/01/rdf-schema#", 
           "owl":"http://www.w3.org/2002/07/owl#",
           "smk":"https://hevs.ch/data/smoking-cessation/",
           "arg" : "https://hevs.ch/ontology/argumentation#", 
           "@language": "en"}
#g.serialize(format='json-ld', context=context, indent=4, destination="../data/arguments.json")
g.serialize(format='turtle', destination="../data/route.ttl")

<Graph identifier=Nb9f8284a990c459dbf2e1c3f05146d9c (<class 'rdflib.graph.Graph'>)>